In [13]:
import torch
import torchvision
from torchvision import transforms
import torch.optim as optim

from sklearn.cluster import KMeans
import json
import numpy as np
import os
from tqdm import tqdm
import time
import random
import torch.nn as nn


In [2]:
# Load dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [3]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)

# Load pre-trained VGG16 model
vgg16 = torchvision.models.vgg16(pretrained=True)
# Modify first layer with 1 channel
vgg16.features[0] = torch.nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
# Remove the last fully connected layer to get feature extraction layer
vgg16.features = torch.nn.Sequential(*list(vgg16.features.children())[:-1])

/home/paul/miniconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/paul/miniconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# Move model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)

# Extracting Features

In [5]:
# Extract features for each image in the dataset
features = []
i = 0
with torch.no_grad():
    for image, label in train_dataset:
        #print(i)

        image = image.to(device)
        features.append(vgg16(image.unsqueeze(0)).cpu().numpy().flatten())
        #labels.append(labels)
        #i = i+1


In [6]:
#takes forever
# # Convert the features and labels to numpy arrays
# features = np.array(features)
# labels = np.array(labels)

# # Save the features and labels to a compressed npz file
# np.savez_compressed('mnist_features_labels.npz', features=features, labels=labels)

In [7]:
# # Load the compressed npz file
# data = np.load('mnist_features_labels.npz')

# # Extract the features and labels from the data dictionary
# features = data['features']
# labels = data['labels']


# Kmeans and Grouping

In [8]:
# Perform k-means clustering with 10 clusters
kmeans = KMeans(n_clusters=10, random_state=42).fit(features)

In [9]:
cluster_labels = []
for i in range(10):
    cluster_indices = np.where(kmeans.labels_ == i)[0]
    cluster_mnist_labels = train_dataset.targets[cluster_indices]
    cluster_labels.append(np.bincount(cluster_mnist_labels).argmax())
#some numbers are not showing
#prob becuz some numbers have similar features with multiple classes
print(cluster_labels)

[7, 0, 4, 3, 2, 8, 0, 7, 1, 5]


In [10]:
clustered_dataset = [[] for i in range(10)]

for i in range(len(features)):
    cluster_label = kmeans.labels_[i] #return the cluster label assigned to the ith dataset
    clustered_dataset[cluster_label].append((train_dataset[i][0], train_dataset[i][1]))


In [11]:
for i in range(10):
    print(f"Cluster{i}: " ,len(clustered_dataset[i]), "datasets")

Cluster0:  4328 datasets
Cluster1:  8039 datasets
Cluster2:  6204 datasets
Cluster3:  3747 datasets
Cluster4:  4631 datasets
Cluster5:  8202 datasets
Cluster6:  6351 datasets
Cluster7:  4156 datasets
Cluster8:  7459 datasets
Cluster9:  6883 datasets


# Testing the Training Time

In [52]:
#for creating custom dataset
from torch.utils.data import Dataset

class DictDataset(Dataset):
    def __init__(self, data_dict, transform=None):
        self.data_dict = data_dict
        self.transform = transform
        self.keys = list(data_dict.keys())
        self.keys.sort()

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, index):
        # Get the image data and label for the current item
        key = self.keys[index]
        img = self.data_dict[key]['images']
        label = self.data_dict[key]['labels']

        if self.transform is not None:
            img = self.transform(img)

        return img, label

Training with the whole dataset 

In [67]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28) # Flatten the input image
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model and optimizer
model = MLP()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Load the train dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

# Define the batch size and number of epochs
batch_size = 128
num_epochs = 10

# Create the dataloader for the train dataset
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [68]:
# Train the model with tqdm progress bar
total_iterations = len(train_loader) * num_epochs
with tqdm(total=total_iterations, desc='Training') as pbar:
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Update the progress bar
            pbar.update(1)

            # Print statistics
            running_loss += loss.item()
            if (i == len(train_loader)-1):
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
                running_loss = 0.0

Training:  10%|███                           | 474/4690 [00:09<01:27, 47.95it/s]

Epoch [1/10], Step [469/469], Loss: 2.0268


Training:  20%|██████                        | 944/4690 [00:18<01:15, 49.69it/s]

Epoch [2/10], Step [469/469], Loss: 0.8375


Training:  30%|████████▋                    | 1413/4690 [00:28<01:04, 50.69it/s]

...

Epoch [8/10], Step [469/469], Loss: 0.2023


Training:  90%|██████████████████████████▏  | 4228/4690 [01:25<00:09, 49.04it/s]

Epoch [9/10], Step [469/469], Loss: 0.1687


Training: 100%|█████████████████████████████| 4690/4690 [01:34<00:00, 49.41it/s]

Epoch [10/10], Step [469/469], Loss: 0.1423


# Custom Dataset

In [70]:
#create dataset with 50% of data
data_dict_50 = {}
index = 0
for i in range(10):
    num_items = len(clustered_dataset[i])
    num_selected = len(clustered_dataset[i])/2
    #select random indices
    selected_indices = random.sample(range(num_items), int(num_selected))
    for j in selected_indices:
        data_dict_50[index] = {'images': clustered_dataset[i][j][0], 'labels': clustered_dataset[i][j][1]}
        index = index+1

In [71]:
len(data_dict_50)

custom_dataset = DictDataset(data_dict_50)

# Initialize the model and optimizer
model1 = MLP()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

# Create the dataloader for the train dataset
train_loader = torch.utils.data.DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

In [72]:

total_iterations = len(train_loader) * num_epochs
with tqdm(total=total_iterations, desc='Training') as pbar:
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model1(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Update the progress bar
            pbar.update(1)

            # Print statistics
            running_loss += loss.item()
            if (i == len(train_loader)-1):
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
                running_loss = 0.0

Training:  11%|███▏                         | 259/2350 [00:02<00:16, 123.98it/s]

Epoch [1/10], Step [235/235], Loss: 1.0306


Training:  21%|██████                       | 496/2350 [00:03<00:13, 136.36it/s]

Epoch [2/10], Step [235/235], Loss: 0.4607


Training:  31%|████████▉                    | 728/2350 [00:05<00:13, 122.94it/s]

...

Epoch [8/10], Step [235/235], Loss: 0.0888


Training:  91%|█████████████████████████▍  | 2139/2350 [00:16<00:01, 131.56it/s]

Epoch [9/10], Step [235/235], Loss: 0.0680


Training: 100%|████████████████████████████| 2350/2350 [00:18<00:00, 126.72it/s]

Epoch [10/10], Step [235/235], Loss: 0.0580


# testing accuracy

In [69]:
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        # Forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test accuracy training with the whole dataset: {100 * correct/total:.2f}%")

Test accuracy training with the whole dataset: 97.60%


In [73]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        # Forward pass
        outputs = model1(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test accuracy for 50% dataset is: {100 * correct/total:.2f}%")

Test accuracy for 50% dataset is: 95.15%
